## Annual Averages

Let's now begin with plotting the annual averages per unit of latitude and longitude. First, we must load the data.

In [1]:
import xarray as xarr

# open datafile
dataFile = "./data/clipped_hourly_pressure_1988.nc"
data = xarr.open_dataset(dataFile)

# kelvin to celsius
data["t_celsius"] = data['t'] - 273.15

Now proceeding to calculate the means and the spatial averages:

In [2]:
# calculate averages across continental US
mean_temp = data['t_celsius'].mean(dim='valid_time')
mean_clwc = data['clwc'].mean(dim='valid_time')

# calculate nationwide average across the year
temp_timeseries = data['t_celsius'].mean(dim=['latitude', 'longitude'])
clwc_timeseries = data['clwc'].mean(dim=['latitude', 'longitude'])

For the spatial averages, we have to use the climate data operators class. Also, for the average cloud water content, a better metric would be the cumulative sum of all waater content across the US. Let's try that now:

In [ ]:
from cdo import Cdo
cdo = Cdo() # load climaate data operators package

# define output file
outputFile = "./data/spatial_avgs.nc"

spatial_avg_file = cdo.fldmean(input=dataFile, output=outputFile)
spatial_avgs = xarr.open_dataset(outputFile)

# calculate nationwide average across the year
temp_timeseries = spatial_avgs["t"] - 273.15 # convert to celsius
clwc_timeseries = spatial_avgs["clwc"].cumsum(dim=["lat", "lon"])

We can then plot the annual average temperature across the continental United States.

In [ ]:
import matplotlib.pyplot as plot
import cartopy.crs as ccrs

# declare figure
theFigure = plot.subplots(figsize=(12, 8), subplot_kw={'projection': ccrs.PlateCarree()})

# make map, adjust colorbar
temp_plot = mean_temp.plot(transform=ccrs.PlateCarree(), cmap="RdYlBu_r", add_colorbar=True)
temp_plot.colorbar.set_label("Temperature (Celsius)")

# plot, show
plot.title("Annual Average Temperature at 1hPa across the Continental United States")
plot.savefig("./plots/Annual_Average_Temperature_Hourly_1hPa_CONUS_1988.png")
plot.show()

Now, let's do the average cloud water content.

In [ ]:
# declare figure
theFigure = plot.subplots(figsize=(12, 8), subplot_kw={'projection': ccrs.PlateCarree()})

# make map, adjust colorbar
clwc_plot = mean_clwc.plot(transform=ccrs.PlateCarree(), cmap="RdYlBu_r", vmin=0, vmax=0.02, add_colorbar=True)
clwc_plot.colorbar.set_label("Cloud Water Content (kg/kg)")

# plot, show
plot.title("Annual Average Cloud Water Content at 1hPa across the Continental United States")
plot.savefig("./plots/Annual_Average_CLWC_Hourly_1hPa_CONUS_1988.png")
plot.show()

(The altitude at which the pressure is 1hPa is so extreme that there is essentially zero cloud water content; hence, the near-uniform graph.)

Let's now plot the annual average across the entire continental United States with a graph that shows how temperature changed with time.

In [ ]:
# create figure
theFigure = plot.subplots(figsize=(12,8))

# need to pass in the times, and then the data
plot.plot(temp_timeseries.valid_time, temp_timeseries, color="red")

# labels
plot.ylabel("Temperature (Celsius)")
plot.xlabel("Date")
plot.title("Average Temperature at 1hPA Across 1988")
plot.savefig("./plots/Annual_Average_Temperature_TimeSeries_Hourly_1hPa_CONUS_1988.png")
plot.show()

We can then finish with the same for cloud water content.

In [ ]:
theFigure = plot.subplots(figsize=(12,8))
plot.plot(clwc_timeseries.valid_time, clwc_timeseries, color="red")
plot.ylabel("Cloud Water Content (kg/kg)")
plot.xlabel("Date")
plot.title("Average Cloud Water Content at 1hPA Across 1988")
plot.ylim(bottom=0)
plot.savefig("./plots/Annual_Average_CLWC_TimeSeries_Hourly_1hPa_CONUS_1988.png")
plot.show()

(Again, at that pressure, it's miniscule.)